In [31]:
import numpy as np
from sklearn.svm import SVR
import torch
from sklearn.model_selection import KFold
from itertools import product
import matplotlib.pyplot as plt

In [22]:
def mean_euclidean_error(vectors1, vectors2):
    """
    Compute the mean Euclidean error between two sets of 3D vectors.

    Parameters:
    - vectors1: NumPy array of shape (N, 3) representing the first set of 3D vectors
    - vectors2: NumPy array of shape (N, 3) representing the second set of 3D vectors

    Returns:
    - mean_error: Mean Euclidean error between the two sets of vectors
    """
    # Check if the input arrays have the correct shape
    if vectors1.shape != vectors2.shape or vectors1.shape[1] != 3:
        raise ValueError("Input arrays must be of shape (N, 3)")

    # Compute Euclidean distance
    euclidean_distance = np.linalg.norm(vectors1 - vectors2, axis=1)

    # Calculate the mean Euclidean error
    mean_error = np.mean(euclidean_distance)

    return mean_error

In [23]:
from sklearn.model_selection import train_test_split
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('ML-CUP23-TR.csv', delimiter=',')
X = dataset[:,1:11]
y = dataset[:,11:14]



# Split the data into training and testing sets (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
class MultiSVM:
    def __init__(self, kernel='rbf', C=1.0, epsilon=0.1):
        # Create three support vector regressors with the specified kernel, regularization parameter, and epsilon
        self.svr0 = SVR(kernel=kernel, C=C, epsilon=epsilon)
        self.svr1 = SVR(kernel=kernel, C=C, epsilon=epsilon)
        self.svr2 = SVR(kernel=kernel, C=C, epsilon=epsilon)

    def fit(self, X, y):
        # Fit each SVR on its respective data
        self.svr0.fit(X, y[:,0])
        self.svr1.fit(X, y[:,1])
        self.svr2.fit(X, y[:,2])

    def predict(self, X):
        # Make predictions using each SVR
        pred = np.column_stack((self.svr0.predict(X),self.svr1.predict(X),self.svr2.predict(X)))
        return pred

In [29]:
def perform_grid_search_kfold(kernels, Cs, epsilons, k_folds, x, y, return_sequence=False):
    """
    Perform grid search with k-fold cross-validation for hyperparameters.

    Parameters:
    - neuron_numbers (list): List of neuron numbers to search.
    - learning_rates (list): List of learning rates to search.
    - momentums (list): List of momentum values to search.
    - batch_sizes (list): List of batch sizes to search.
    - reg_coeffs (list): List of regularization coefficients to search.
    - activations (list): List of activation functions to search.
    - layerss (list): List of numbers of hidden layers to search.
    - k_folds (int): Number of folds for cross-validation.
    - x (numpy.ndarray): Input data.
    - y (numpy.ndarray): Target data.
    - plot_curves (bool, optional): Whether to plot training curves (default: False).
    - num_epochs (int, optional): Number of training epochs (default: 1000).

    Returns:
    - list: List of best hyperparameters.

    The function performs grid search with k-fold cross-validation for Monk classifier hyperparameters and returns the best hyperparameters.
    """
    mee_sequence = []
    mee_sd_sequence = []
    best_mee = float('inf')
    best_mee_std = 0
    best_hyperparams = []
    counter = 0
    num_combinations = sum(1 for _ in product(kernels, Cs,epsilons))
    print('total number of grid search combinations explored:',num_combinations)
    for kernel, C,epsilon in product(kernels, Cs,epsilons):
        counter += 1
        print(f'{counter}/{num_combinations} Hyperparams:',kernel, C, epsilon)

        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
        val_mees = []

        # Perform K-fold cross-validation
        for fold, (train_indices, val_indices) in enumerate(kf.split(x,y)):
            #print(f"\nFold {fold + 1}/{k_folds}")

            # Split the data into training and validation (or test) sets
            X_train, X_val = x[train_indices], x[val_indices]
            y_train, y_val = y[train_indices], y[val_indices]

            model = MultiSVM(kernel, C, epsilon)
            model.fit(X_train,y_train)
            val_mees.append(mean_euclidean_error(model.predict(X_val),y_val))

        print(f'Final Results: kernel={kernel}; C={C}; epsilon={epsilon} --> '
            f'val_mee = {np.mean(val_mees):.4} +- {np.std(val_mees):.4}')

        mee_sequence.append(np.mean(val_mees))
        mee_sd_sequence.append(np.std(val_mees))
        if np.mean(val_mees) < best_mee:
            best_mee = np.mean(val_mees)
            best_mee_std = np.std(val_mees)
            best_hyperparams = [kernel, C, epsilon]

    print(f'Best Hp: {best_hyperparams} with MEE = {best_mee} +- {best_mee_std}')
    if return_sequence:
        return best_hyperparams, mee_sequence, mee_sd_sequence
    else:
        return best_hyperparams

In [26]:
'''
#first coarse grid search
kernels = ['rbf']
Cs = [0.01,0.1,1,10,100,1000]
epsilons = [0.01,0.1,1,10]
#Best Hp: ['rbf', 1000, 0.1] with MEE = 0.6396615599226871 +- 0.04980493197143183
'''

'''
#finer grid search
kernels = ['rbf']
Cs = [1000,2000,3000,4000,5000]
epsilons = [0.1,0.2,0.3,0.4,0.5]
# Best Hp: ['rbf', 3000, 0.1] with MEE = 0.6172759223965806 +- 0.06576969897515686
'''

best_hyperparams = perform_grid_search_kfold(kernels,
                          Cs,
                          epsilons,
                          k_folds=3,
                          x=X_train,
                          y=y_train)

total number of grid search combinations explored: 25
1/25 Hyperparams: rbf 1000 0.1
Final Results: kernel=rbf; C=1000; epsilon=0.1 --> val_mee = 0.6397 +- 0.0498
2/25 Hyperparams: rbf 1000 0.2
Final Results: kernel=rbf; C=1000; epsilon=0.2 --> val_mee = 0.6635 +- 0.04859
3/25 Hyperparams: rbf 1000 0.3
Final Results: kernel=rbf; C=1000; epsilon=0.3 --> val_mee = 0.7216 +- 0.05159
4/25 Hyperparams: rbf 1000 0.4
Final Results: kernel=rbf; C=1000; epsilon=0.4 --> val_mee = 0.7893 +- 0.04696
5/25 Hyperparams: rbf 1000 0.5
Final Results: kernel=rbf; C=1000; epsilon=0.5 --> val_mee = 0.8715 +- 0.04746
6/25 Hyperparams: rbf 2000 0.1
Final Results: kernel=rbf; C=2000; epsilon=0.1 --> val_mee = 0.6185 +- 0.0589
7/25 Hyperparams: rbf 2000 0.2
Final Results: kernel=rbf; C=2000; epsilon=0.2 --> val_mee = 0.65 +- 0.05142
8/25 Hyperparams: rbf 2000 0.3
Final Results: kernel=rbf; C=2000; epsilon=0.3 --> val_mee = 0.7114 +- 0.05378
9/25 Hyperparams: rbf 2000 0.4
Final Results: kernel=rbf; C=2000; epsi

Study the effect of the parameter C

In [30]:
kernels = ['rbf']
Cs = [10,20,30,40]
epsilons = [0.1]


best_hyperparams, mees, mees_sd = perform_grid_search_kfold(kernels,
                          Cs,
                          epsilons,
                          k_folds=3,
                          x=X_train,
                          y=y_train,
                          return_sequence=True)

total number of grid search combinations explored: 4
1/4 Hyperparams: rbf 10 0.1
Final Results: kernel=rbf; C=10; epsilon=0.1 --> val_mee = 1.609 +- 0.08875
2/4 Hyperparams: rbf 20 0.1
Final Results: kernel=rbf; C=20; epsilon=0.1 --> val_mee = 1.375 +- 0.08206
3/4 Hyperparams: rbf 30 0.1
Final Results: kernel=rbf; C=30; epsilon=0.1 --> val_mee = 1.255 +- 0.0801
4/4 Hyperparams: rbf 40 0.1
Final Results: kernel=rbf; C=40; epsilon=0.1 --> val_mee = 1.172 +- 0.07708
Best Hp: ['rbf', 40, 0.1] with MEE = 1.172096760255979 +- 0.07707830189581184


In [ ]:
def plot_mean_std(cs,mee,std,label):

    plt.figure(figsize=(9, 8))
    plt.subplot(2,1,1)
    plt.plot(mean_tr, label=f'Training {label} (mean $\pm$ std)', color = 'red', linewidth=1)
    plt.fill_between(range(0,len(train_hist[0])),mean_tr-std_tr, mean_tr+std_tr, color='crimson', alpha=0.3)

    plt.plot(mean_te, label=f'Test {label} (mean $\pm$ std)', color = 'blue', linestyle='--', linewidth=1)
    plt.fill_between(range(0,len(test_hist[0])),mean_te-std_te, mean_te+std_te, color='blue', alpha=0.3)

    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.legend()



    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.ylim(0,1)
    plt.legend()
    plt.show()


In [ ]:
model = MultiSVM(*best_hyperparams)
model.fit(X_train,y_train)
print(mean_euclidean_error(model.predict(X_test),y_test))

0.5050968675207652
